In [1]:
# -*- coding: utf-8 -*-
#  Stage 2: Data Curation

import os
import pandas as pd
import shutil
import glob   # للبحث عن الملفات تلقائياً
from IPython.display import display  # للعرض داخل Colab

# تحديد مسارات مناطق البيانات
processed_zone_path = 'data_lake/processed_zone'
curated_zone_path = 'data_lake/curated_zone'

# إنشاء المجلدات (إذا لم تكن موجودة)
os.makedirs(processed_zone_path, exist_ok=True)
os.makedirs(curated_zone_path, exist_ok=True)

print(" Directories ready:")
print(f" - Processed Zone: {processed_zone_path}")
print(f" - Curated Zone: {curated_zone_path}")


 Directories ready:
 - Processed Zone: data_lake/processed_zone
 - Curated Zone: data_lake/curated_zone


In [2]:
from google.colab import files

# رفع الملف الناتج من المرحلة السابقة (students_processed.parquet)
uploaded = files.upload()

if uploaded:
    # استخراج اسم الملف الذي تم رفعه
    uploaded_file_name = list(uploaded.keys())[0]

    # تحديد الوجهة في processed_zone
    dst = os.path.join(processed_zone_path, uploaded_file_name)

    # نقل الملف للمكان الصحيح
    shutil.move(uploaded_file_name, dst)
    print(f" تم نقل الملف بنجاح إلى: {dst}")
else:
    print(" لم يتم رفع أي ملف.")


Saving students_processed.parquet to students_processed.parquet
 تم نقل الملف بنجاح إلى: data_lake/processed_zone/students_processed.parquet


In [3]:
print("[1/3] Loading processed data from the Processed Zone...")

# استخدام glob للعثور على ملفات parquet
parquet_files = glob.glob(os.path.join(processed_zone_path, '*.parquet'))

if not parquet_files:
    print(f" ERROR: لا يوجد أي ملفات Parquet في {processed_zone_path}")
    print(" شغل مرحلة الـ Processing أولًا.")
    raise FileNotFoundError("No Parquet files found.")

# اختيار أول ملف
processed_file = parquet_files[0]
print(f"📄 Found and loading file: {processed_file}")

# قراءة الملف
df = pd.read_parquet(processed_file)
print(" Processed data loaded successfully.")


[1/3] Loading processed data from the Processed Zone...
📄 Found and loading file: data_lake/processed_zone/students_processed.parquet
 Processed data loaded successfully.


In [4]:
print("\n[2/3] Data Curation: Performing aggregations...")

#  التجميع 1: متوسط الدرجات لكل مادة
avg_score_per_subject = (
    df.groupby('subject_name')['score']
    .mean()
    .reset_index()
    .round(2)
)
avg_score_per_subject.rename(columns={'score': 'average_score'}, inplace=True)

#  التجميع 2: الأداء حسب المدينة
performance_by_city = (
    df.groupby('city')
    .agg(
        average_attendance_rate=('attendance_rate', 'mean'),
        average_gpa=('previous_gpa', 'mean'),
        student_count=('student_id', 'count')
    )
    .reset_index()
    .round(2)
)

# عرض النتائج
print("\n Average Score per Subject:")
display(avg_score_per_subject.head(10))

print("\n Performance Summary by City:")
display(performance_by_city.head(10))



[2/3] Data Curation: Performing aggregations...

 Average Score per Subject:


subject_name  average_score
0       arabic          85.53
1      english          85.97
2      history          82.46
3         math          82.68
4      science          75.46


 Performance Summary by City:


city  average_attendance_rate  average_gpa  student_count
0  alexandria                    81.78         2.86            192
1       cairo                    81.49         2.93            197
2        giza                    83.09         2.94            182
3       luxor                    82.78         2.92            225
4    mansoura                    81.05         2.88            204

In [5]:
print("\n[3/3] Saving curated datasets to the Curated Zone...")

# حفظ نتائج التجميع في ملفات CSV
curated_file_1 = os.path.join(curated_zone_path, 'avg_score_per_subject.csv')
avg_score_per_subject.to_csv(curated_file_1, index=False)

curated_file_2 = os.path.join(curated_zone_path, 'performance_by_city.csv')
performance_by_city.to_csv(curated_file_2, index=False)

print(f" Saved curated file 1 to: {curated_file_1}")
print(f" Saved curated file 2 to: {curated_file_2}")

print("\n Stage 2 Complete! Curated data is ready for analysis.")



[3/3] Saving curated datasets to the Curated Zone...
 Saved curated file 1 to: data_lake/curated_zone/avg_score_per_subject.csv
 Saved curated file 2 to: data_lake/curated_zone/performance_by_city.csv

 Stage 2 Complete! Curated data is ready for analysis.


In [6]:
from google.colab import files

# تحميل الملفات الناتجة
files.download('data_lake/curated_zone/avg_score_per_subject.csv')
files.download('data_lake/curated_zone/performance_by_city.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
from google.colab import files

files.download(curated_file_1)
files.download(curated_file_2)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>